In [1]:
# articleUrl
# articleSeq //불러온 첫번째 seq가 원래꺼보다 클 때
# articleRegDate
# price
# title
# sortDate
# state # 0 1 3

In [2]:
import pandas as pd
import json
import requests
import time
from tqdm import tqdm

한번에 최대 10000개

3일 단위로 계속 불러오기 6개월까지

In [3]:
query="아이폰 13"

url = f'https://search-api.joongna.com/v25/search/product'
param = {
    "filter":{
        "dateFilterParameter":{
            "sortEndDate":"2022-04-13",
            "sortStartDate":"2022-04-12"
        },
        "flawedYn":0,
        "fullPackageYn":0,
        "limitedEditionYn":0,
#         "maxPrice":2000000000,
#         "minPrice":2,
        "tradeType":0
    },
    "productFilter":"ALL",
    "productStates":[
        0,
        1,
        99
    ],
    "searchQuantity":10000,
    "osType":2,
    "searchWord":f'{query}',
    "sort":"RECENT_SORT",
}

In [21]:
#json.dumps
response= requests.post(url,json=(param))
data=response.json()['data']
len(data['items'])

2592

In [33]:
def make_binary(x):
    if x!=0:
        return 1
    return 0

col=['articleUrl',
'articleSeq',
'articleRegDate',
'price',
'title',
'sortDate',
'state'] # 0 1 3]
df=pd.DataFrame(data['items']).drop_duplicates('title')[col]
df=df.dropna()
df.rename(columns  = {'state': 'is_sold'}, inplace = True)
df['is_sold'] = df['is_sold'].map(lambda x : make_binary(x))
df['articleSeq']=df['articleSeq'].astype('int')

In [34]:
detail_list=[]
for seq in tqdm(df[df['is_sold']==0]['articleSeq']):
    t=time.time()
    detail_url=f'https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10050146/articles/{seq}'
    response=requests.get(detail_url)
    
    if response.status_code!=200:
        continue
    detail=response.json()
    city=detail['result']['articleRegion']['regionName1']
    state=detail['result']['articleRegion']['regionName2']
    img=detail['result']['saleInfo']['imgUrl']
    detail_list.append([seq,city,state])

detail_df=pd.DataFrame(detail_list,columns=["articleSeq",'city','state'])

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:48<00:00,  7.55it/s]


In [35]:
total=pd.merge(df,detail_df, how='outer',on='articleSeq')
total=total.fillna("")
total

,articleUrl,articleSeq,articleRegDate,price,title,sortDate,is_sold,city,state
0,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608326,2022-04-13 23:59:58,2999000,"아이폰13 프로/프로맥스 pro max 당일매입(자급제,당일개통)",2022-04-13 23:59:58,0,,
1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608321,2022-04-13 23:59:55,1165000,애플 아이폰13 프로 128g 알파인그린 판매합니다,2022-04-13 23:59:55,0,NaN,NaN
2,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608275,2022-04-13 23:59:31,4999999,"아이폰13 PRO MAX 프로 맥스 자급제미개봉,당일개통,박스폰삽니다",2022-04-13 23:59:31,0,,
3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608191,2022-04-13 23:58:49,1550000,아이폰13 프로 맥스 갤럭시S22 울트라 전기종 삽니다. 자급제 통신사 유심기변,2022-04-13 23:58:49,0,,
4,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608163,2022-04-13 23:58:32,1000000,"[매입]아이폰13,아이폰전기종 자급제,박스폰,중고폰,파손폰 당일매입",2022-04-13 23:58:32,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
424,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910240970,2022-04-12 10:34:19,65000,케이스티파이 아이폰13프로맥스 맥세이프 임팩트 케이스 판매합니다.,2022-04-12 10:34:19,1,NaN,NaN
425,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910223599,2022-04-12 09:13:23,1220000,아이폰 13프로 골드 256 애케플 가입팝니다,2022-04-12 09:13:23,1,NaN,NaN
426,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910217357,2022-04-12 08:37:22,40000,아이폰13 PRO 프레임바이 에어/에어범퍼 케이스 팝니다.,2022-04-12 08:37:22,1,NaN,NaN
427,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910216441,2022-04-12 08:31:42,40000,아이폰13 PRO 케이스티파이 Duty Free 임팩트,2022-04-12 08:31:42,1,NaN,NaN


In [65]:
no_buy = df[df['title'].str.contains('삽니|매입|사요', na=False)]
# pd.merge(total,no_buy,how="left")
only_sale=pd.merge(total, no_buy, how='outer', indicator = True).query('_merge == "left_only"').drop(columns=['_merge'])
only_sale

,articleUrl,articleSeq,articleRegDate,price,title,sortDate,is_sold,city,state
1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608321,2022-04-13 23:59:55,1165000,애플 아이폰13 프로 128g 알파인그린 판매합니다,2022-04-13 23:59:55,0,,
6,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910607899,2022-04-13 23:56:10,100000,아더에러 케이스티파이 아이폰13프로 판매합니다.,2022-04-13 23:56:10,0,,
11,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910606434,2022-04-13 23:43:36,700000,아이폰13미니 블랙색상 128용량 배터리100% 새상품컨디션 초S급 70만 팝니다,2022-04-13 23:43:36,0,,
13,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910606177,2022-04-13 23:41:30,2000,(195991)아이폰13미니 128기가 핑크 SS급 70만원,2022-04-13 23:41:30,0,,
14,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910605874,2022-04-13 23:38:51,980000,아이폰13pro 팝니다,2022-04-13 23:38:51,0,,
...,...,...,...,...,...,...,...,...,...
424,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910240970,2022-04-12 10:34:19,65000,케이스티파이 아이폰13프로맥스 맥세이프 임팩트 케이스 판매합니다.,2022-04-12 10:34:19,1,,
425,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910223599,2022-04-12 09:13:23,1220000,아이폰 13프로 골드 256 애케플 가입팝니다,2022-04-12 09:13:23,1,,
426,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910217357,2022-04-12 08:37:22,40000,아이폰13 PRO 프레임바이 에어/에어범퍼 케이스 팝니다.,2022-04-12 08:37:22,1,,
427,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910216441,2022-04-12 08:31:42,40000,아이폰13 PRO 케이스티파이 Duty Free 임팩트,2022-04-12 08:31:42,1,,
